In [2]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
print(tf.__version__)

2.8.0


In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

column_names = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare',
       'class', 'deck', 'embark_town', 'alone']

df = pd.read_csv(TRAIN_DATA_URL)
# pd.set_option('display.max_rows', None)
print(df)
# 데이터 정보 확인
print(df.info())
# 수치형 데이터 확인
print(df.describe())
# 범주형 데이터 확인
df.describe(include = np.object_)
# 결측치 확인
df.isnull().sum()

     survived     sex   age  n_siblings_spouses  parch     fare   class  \
0           0    male  22.0                   1      0   7.2500   Third   
1           1  female  38.0                   1      0  71.2833   First   
2           1  female  26.0                   0      0   7.9250   Third   
3           1  female  35.0                   1      0  53.1000   First   
4           0    male  28.0                   0      0   8.4583   Third   
..        ...     ...   ...                 ...    ...      ...     ...   
622         0    male  28.0                   0      0  10.5000  Second   
623         0    male  25.0                   0      0   7.0500   Third   
624         1  female  19.0                   0      0  30.0000   First   
625         0  female  28.0                   1      2  23.4500   Third   
626         0    male  32.0                   0      0   7.7500   Third   

        deck  embark_town alone  
0    unknown  Southampton     n  
1          C    Cherbourg     n

survived              0
sex                   0
age                   0
n_siblings_spouses    0
parch                 0
fare                  0
class                 0
deck                  0
embark_town           0
alone                 0
dtype: int64

In [4]:
# 데이터 요약

print("전체 데이터 수:", df.shape[0] * df.shape[1])
print(f"결측치 수: {df.isnull().sum().sum()}")
print("총 인원 수:", df["age"].count())
print("중복된 데이터:",df.duplicated().sum())

전체 데이터 수: 6270
결측치 수: 0
총 인원 수: 627
중복된 데이터: 69


In [16]:
class_names = ['UnSurvived', 'Survived']

feature_names = column_names[1:]
label_name = column_names[0]

print(feature_names)
print(label_name)

batch_size = 32
train_dataset = tf.data.experimental.make_csv_dataset(
    train_file_path,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

test_dataset = tf.data.experimental.make_csv_dataset(
    train_file_path,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

features, labels = next(iter(train_dataset))

print(f"##features \n {features}")


['sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
survived
##features 
 OrderedDict([('sex', <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'male', b'female', b'male', b'male', b'male', b'male', b'male',
       b'female', b'male', b'male', b'male', b'male', b'male', b'male',
       b'male', b'female', b'male', b'male', b'male', b'female', b'male',
       b'male', b'male', b'male', b'male', b'male', b'female', b'male',
       b'female', b'male', b'male', b'female'], dtype=object)>), ('age', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([18. , 45. , 30.5, 28. , 19. , 22. , 28. ,  2. , 28. , 24. , 28. ,
       23. , 15. , 48. , 31. , 26. , 41. , 32. , 28. , 23. , 20. , 35. ,
       46. , 28. , 31. , 26. , 49. , 29. , 32. , 71. , 28. , 52. ],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0,

In [17]:
# 불필요한 컬럼 삭제 -> embark_town
def remove_columns(features, labels):
    del(features['embark_town'])
    return features, labels

train_dataset = train_dataset.map(remove_columns)
test_dataset = test_dataset.map(remove_columns)
features, labels = next(iter(train_dataset))

print(f"features \n {features}")

features 
 OrderedDict([('sex', <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'male', b'male', b'male', b'male', b'male', b'female', b'female',
       b'male', b'male', b'male', b'female', b'male', b'female', b'male',
       b'male', b'male', b'female', b'male', b'female', b'female',
       b'male', b'male', b'male', b'male', b'male', b'male', b'male',
       b'female', b'female', b'female', b'male', b'male'], dtype=object)>), ('age', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([16. , 61. , 50. , 22. , 56. , 11. , 15. , 17. , 39. , 28. , 19. ,
       32. , 42. , 22. , 34. , 21. , 28. , 28. , 17. , 18. , 20. , 19. ,
       28.5, 28. , 33. , 25. , 44. , 28. , 14. , 14. , 36. , 28. ],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([4, 0, 1, 1, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0], dtype=int32)>), ('parch', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 0, 0, 0,

In [18]:
# 문자열 처리 -> sex, deck, alone, class를 문자열로
def convert_to_int(feature, label):
    if feature['sex'] == 'male':
        feature['sex'] = 0.
    else:
        feature['sex'] = 1.
    
    feature['sex'] = tf.cast(feature['sex'], tf.float32)
    
    return feature, label

CAT_COLUMNS = ['sex', 'deck', 'alone', 'class']
NUM_COLUMNS = ['age', 'fare', 'n_siblings_spouses', 'parch']

feature_cols = []

# Create IndicatorColumn for categorical features
for feature in CAT_COLUMNS:
  vocab = df[feature].unique()
  feature_cols.append(tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature, vocab)))

# Create NumericColumn for numerical features
for feature in NUM_COLUMNS:
  feature_cols.append(tf.feature_column.numeric_column(feature, dtype=tf.float32))

print(feature_cols)

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, n

In [15]:
# 모델 생성
model = tf.keras.Sequential()
model.add(tf.keras.layers.DenseFeatures(feature_cols))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])

model.fit(train_dataset, epochs=100)
        

Epoch 1/1000


2022-05-15 12:15:11.331207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 1s 26ms/step - loss: 0.7944 - binary_accuracy: 0.6762
Epoch 2/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.6728 - binary_accuracy: 0.6730
Epoch 3/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.6472 - binary_accuracy: 0.6715
Epoch 4/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.6188 - binary_accuracy: 0.6890
Epoch 5/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.6241 - binary_accuracy: 0.6842
Epoch 6/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.6162 - binary_accuracy: 0.6810
Epoch 7/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.6103 - binary_accuracy: 0.7002
Epoch 8/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.6075 - binary_accuracy: 0.6906
Epoch 9/1000
20/20 [==============================] - 0s 8ms/step - loss: 0.6427 - binary_accuracy: 0.6746
Epoch 10/1000
20/20 [===========================

20/20 [==============================] - 0s 9ms/step - loss: 0.5178 - binary_accuracy: 0.7671
Epoch 77/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5411 - binary_accuracy: 0.7448
Epoch 78/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5269 - binary_accuracy: 0.7592
Epoch 79/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5363 - binary_accuracy: 0.7432
Epoch 80/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5677 - binary_accuracy: 0.7049
Epoch 81/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5107 - binary_accuracy: 0.7799
Epoch 82/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5505 - binary_accuracy: 0.7496
Epoch 83/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5404 - binary_accuracy: 0.7480
Epoch 84/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5327 - binary_accuracy: 0.7671
Epoch 85/1000
20/20 [=====================

20/20 [==============================] - 0s 10ms/step - loss: 0.5641 - binary_accuracy: 0.7177
Epoch 152/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5464 - binary_accuracy: 0.7129
Epoch 153/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5907 - binary_accuracy: 0.6906
Epoch 154/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5483 - binary_accuracy: 0.7241
Epoch 155/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.5696 - binary_accuracy: 0.7065
Epoch 156/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.5477 - binary_accuracy: 0.7049
Epoch 157/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5495 - binary_accuracy: 0.7257
Epoch 158/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5572 - binary_accuracy: 0.7097
Epoch 159/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.5591 - binary_accuracy: 0.7065
Epoch 160/1000
20/20 [=========

...

20/20 [==============================] - 0s 9ms/step - loss: 0.4400 - binary_accuracy: 0.8070
Epoch 677/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4536 - binary_accuracy: 0.7959
Epoch 678/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4255 - binary_accuracy: 0.8054
Epoch 679/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4166 - binary_accuracy: 0.8038
Epoch 680/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4237 - binary_accuracy: 0.8022
Epoch 681/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4449 - binary_accuracy: 0.8006
Epoch 682/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4314 - binary_accuracy: 0.8166
Epoch 683/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4590 - binary_accuracy: 0.8054
Epoch 684/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4190 - binary_accuracy: 0.8166
Epoch 685/1000
20/20 [===========

20/20 [==============================] - 0s 9ms/step - loss: 0.4644 - binary_accuracy: 0.8054
Epoch 752/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4234 - binary_accuracy: 0.8166
Epoch 753/1000
20/20 [==============================] - 0s 11ms/step - loss: 0.4247 - binary_accuracy: 0.8102
Epoch 754/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4629 - binary_accuracy: 0.7879
Epoch 755/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4402 - binary_accuracy: 0.8070
Epoch 756/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4470 - binary_accuracy: 0.7959
Epoch 757/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4206 - binary_accuracy: 0.8150
Epoch 758/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4540 - binary_accuracy: 0.8006
Epoch 759/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4494 - binary_accuracy: 0.8006
Epoch 760/1000
20/20 [=======

20/20 [==============================] - 0s 8ms/step - loss: 0.4224 - binary_accuracy: 0.8086
Epoch 827/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4250 - binary_accuracy: 0.8070
Epoch 828/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4283 - binary_accuracy: 0.8150
Epoch 829/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4340 - binary_accuracy: 0.8150
Epoch 830/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4307 - binary_accuracy: 0.8006
Epoch 831/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4222 - binary_accuracy: 0.8038
Epoch 832/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4090 - binary_accuracy: 0.8214
Epoch 833/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4250 - binary_accuracy: 0.8118
Epoch 834/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4161 - binary_accuracy: 0.8102
Epoch 835/1000
20/20 [============

20/20 [==============================] - 0s 9ms/step - loss: 0.4399 - binary_accuracy: 0.8006
Epoch 902/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4455 - binary_accuracy: 0.8102
Epoch 903/1000
20/20 [==============================] - 0s 12ms/step - loss: 0.4658 - binary_accuracy: 0.7974
Epoch 904/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4375 - binary_accuracy: 0.8054
Epoch 905/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4322 - binary_accuracy: 0.8070
Epoch 906/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4130 - binary_accuracy: 0.8166
Epoch 907/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4242 - binary_accuracy: 0.7990
Epoch 908/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4243 - binary_accuracy: 0.8150
Epoch 909/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4236 - binary_accuracy: 0.8086
Epoch 910/1000
20/20 [========

20/20 [==============================] - 0s 10ms/step - loss: 0.4322 - binary_accuracy: 0.8070
Epoch 977/1000
20/20 [==============================] - 0s 11ms/step - loss: 0.4113 - binary_accuracy: 0.8166
Epoch 978/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4094 - binary_accuracy: 0.8134
Epoch 979/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4225 - binary_accuracy: 0.8182
Epoch 980/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4121 - binary_accuracy: 0.8278
Epoch 981/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4307 - binary_accuracy: 0.8198
Epoch 982/1000
20/20 [==============================] - 0s 9ms/step - loss: 0.4080 - binary_accuracy: 0.8150
Epoch 983/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4257 - binary_accuracy: 0.8150
Epoch 984/1000
20/20 [==============================] - 0s 10ms/step - loss: 0.4549 - binary_accuracy: 0.8054
Epoch 985/1000
20/20 [=======

In [25]:
# 모델 평가하기
model.predict(test_dataset)

array([[2.60473400e-01],
       [6.75309062e-01],
       [1.04401380e-01],
       [2.50850320e-01],
       [5.20708025e-01],
       [4.86050874e-01],
       [8.56431127e-02],
       [1.01507813e-01],
       [9.22640562e-02],
       [1.38402800e-03],
       [1.11529566e-01],
       [1.28538430e-01],
       [7.74761587e-02],
       [6.14254177e-02],
       [4.30676401e-01],
       [7.22485632e-02],
       [8.81086215e-02],
       [9.81070638e-01],
       [9.77286920e-02],
       [1.44205049e-01],
       [3.65358144e-01],
       [1.82940528e-01],
       [9.09213871e-02],
       [3.66293639e-01],
       [2.58289844e-01],
       [9.85048592e-01],
       [1.01928040e-01],
       [2.69071698e-01],
       [7.64722526e-01],
       [4.50708091e-01],
       [6.20859683e-01],
       [8.82937685e-02],
       [1.12025127e-01],
       [1.62650406e-01],
       [1.28505141e-01],
       [9.90591586e-01],
       [1.93070680e-01],
       [8.72188956e-02],
       [9.40835550e-02],
       [8.70207623e-02],
